Run me in your browser at [https://colab.research.google.com/github/ds-sloth/TheXTech3DS/blob/master/ep_convert_test.ipynb](https://colab.research.google.com/github/ds-sloth/TheXTech3DS/blob/master/ep_convert_test.ipynb)!

# Build and install conversion tools

In [ ]:
!apt install imagemagick libmagick++-dev libfftw3-dev ffmpeg git automake dtrx
!git clone "https://github.com/devkitPro/3dstools" 3dstools; cd 3dstools; ./autogen.sh; ./configure; make; mv mkromfs3ds /usr/local/bin; cd ..; rm 3dstools -rf
!git clone "https://github.com/devkitPro/tex3ds" tex3ds; cd tex3ds; ./autogen.sh; ./configure; make; mv tex3ds /usr/local/bin; cd ..; rm tex3ds -rf

# Upload an archive file containing your episode(s)

In [ ]:
from google.colab import files
uploaded = files.upload()
filename = None
# find the first item that just got uploaded
for filename in uploaded.keys():
    break
if filename is None:
    raise RuntimeError("No file supplied; please repeat.")
!mkdir uploaded; cd uploaded; dtrx --one here -f "../$filename"; rm "../$filename"

# figure out which nesting level contains episodes
import os
import shutil
episode_folder = 'uploaded'
episode_found = False
while not episode_found:
    for i in os.listdir(episode_folder):
        if i.endswith('.wld') or i.endswith('.wldx'):
            episode_found = True
            break
    if not episode_found:
        for i in os.listdir(episode_folder):
            if os.path.isdir(os.path.join(episode_folder, i)):
                episode_folder = os.path.join(episode_folder, i)
                break
        # if no subfolders were found, and this is not an episode
        else:
            raise RuntimeError("Could not find episode directory -- ensure you have uploaded an episode folder or a folder containing only episode folders.")

if episode_folder == 'uploaded':
    os.mkdir('episodes')
    shutil.move('uploaded', 'episodes/episode')
else:
    shutil.move(episode_folder[:episode_folder.rfind('/')], 'episodes')

# Convert your episode to 3DS format!

In [ ]:
#!/usr/bin/python3

# this is a simple script

import os
import sys
import shutil
import tqdm

PREVIEW = False
REDO = False
datadir = 'episodes'
graphicsdir = os.path.join(datadir, 'graphics')
outdir = 'converted'

for dirpath, _, files in tqdm.tqdm(list(os.walk(datadir, topdown=True))):
    outpath = outdir+dirpath[len(datadir):]
    os.makedirs(outpath, exist_ok=True)
    for fn in files:
        rfn = os.path.join(dirpath, fn)
        if not os.path.isfile(rfn): continue
        destfn = os.path.join(outpath, fn)
        bmpfn = os.path.join(outpath, fn[:-3]+'bmp')
        t3xfn = os.path.join(outpath, fn[:-3]+'png')
        if not REDO and (os.path.isfile(destfn) or ((fn.endswith('.gif') or fn.endswith('.png')) and os.path.isfile(t3xfn))): continue
        #print(rfn)
        if fn.endswith('.png'):
            os.system(f'convert -sample 50% "{rfn}" "{bmpfn}"')
        elif fn.endswith('m.gif') and os.path.isfile(rfn[:-5]+'.gif'):
            continue
        elif fn.endswith('.gif'):
            maskfn = rfn[:-4]+'m.gif'
            ftype = fn[:fn.rfind('-')]
            altmaskfn_gif = os.path.join(graphicsdir, ftype, fn[:-4]+'m.gif')
            altmaskfn_png = os.path.join(graphicsdir, ftype, fn[:-4]+'.png')
            if os.path.isfile(maskfn):
                os.system(f'convert "{rfn}" "{maskfn}" -alpha Off -compose CopyOpacity -composite -channel a -negate +channel -sample 50% -compress None "{bmpfn}"')
            elif os.path.isfile(altmaskfn_gif):
                if os.popen(f'identify -format "%[fx:w*2],%[fx:h*2]" "{rfn}"').read() == os.popen(f'identify -format "%[fx:w*2],%[fx:h*2]" "{altmaskfn_gif}"').read():
                    os.system(f'convert "{rfn}" "{altmaskfn_gif}" -alpha Off -compose CopyOpacity -composite -channel a -negate +channel -sample 50% -compress None "{bmpfn}"')
                else:
                    os.system(f'convert -sample 50% -compress None "{rfn}" "{bmpfn}"')
            elif os.path.isfile(altmaskfn_png):
                if os.popen(f'identify -format "%[fx:w*2],%[fx:h*2]" "{rfn}"').read() == os.popen(f'identify -format "%[fx:w*2],%[fx:h*2]" "{altmaskfn_png}"').read():
                    os.system(f'convert "{rfn}" "{altmaskfn_png}" -alpha On -compose CopyOpacity -composite -sample 50% -compress None "{bmpfn}"')
                else:
                    os.system(f'convert -sample 50% -compress None "{rfn}" "{bmpfn}"')
            else:
                os.system(f'convert -sample 50% -compress None "{rfn}" "{bmpfn}"')
        elif fn.endswith('.db'):
            continue
        elif fn.endswith('.ogg'): continue
        elif fn.endswith('.mp3'):
            os.system(f'ffmpeg -i "{rfn}" -aq 1 "{destfn}.ogg"')
            shutil.move(destfn+'.ogg', destfn)
            continue
        else:
            shutil.copy(rfn, destfn)
            continue
        _output = os.popen(f'identify -format "%[fx:w*2],%[fx:h*2]" "{bmpfn}"').read()
        w, h = _output.split(',')
        open(t3xfn+'.size','w').write(f'{w:>4}\n{h:>4}\n')
        t3xfns = [t3xfn]
        bmpfns = [bmpfn]
        if int(h) > 2048:
            os.system(f'convert "{bmpfn}" -crop {w}x1024 -compress None "{bmpfn}%d.bmp"')
            os.remove(bmpfn)
            shutil.move(bmpfn+'0.bmp', bmpfn) #???
            t3xfns.append(t3xfn+'1')
            bmpfns.append(bmpfn+'1.bmp')
            if int(h) > 4096:
                t3xfns.append(t3xfn+'2')
                bmpfns.append(bmpfn+'2.bmp')
        for t3xfn_i, bmpfn_i in zip(t3xfns, bmpfns):
            if PREVIEW:
                pvwfn_i = t3xfn_i+'.bmp'
                os.system(f'tex3ds "{bmpfn_i}" -f rgba8888 -o "{t3xfn_i}" -p "{pvwfn_i}"')
            else:
                if os.system(f'tex3ds "{bmpfn_i}" -f rgba8888 -o "{t3xfn_i}"'):
                    print(f'During call `tex3ds "{bmpfn_i}" -f rgba8888 -o "{t3xfn_i}"`,')
                    print(f"    something didn't work and {t3xfn_i} is missing. (Size: {w}x{h})")
                os.remove(bmpfn_i)

# Download your newly converted episode(s)

Save the romfs package to /3ds/thextech on your SD card. You may store multiple episode packages with different filenames so long as they have the .romfs file extension.

In [ ]:
!mkromfs3ds converted converted.romfs
from google.colab import files
files.download('converted.romfs')